In [3]:
import pandas as pd
import numpy as np


In [4]:
steam_games = pd.read_csv('steam_games_all_fields.csv', index_col=0, delimiter='\t')

# Split out 'appid' from the URL string.  This will be used later to merge in my own games
# from the steam api
app_id = steam_games['steam_url'].str.split('/',expand=True)
app_id = app_id.drop([0, 1, 2, 5, 6], axis=1)
app_id.columns = ['app', 'game_ID']
df = pd.concat([steam_games, app_id], axis=1, sort=False)

# Fill nil values and remove random characters
df['description'] = df['description'].fillna('')
df = df[~df.description.str.contains('no description')]
df['release_date'] = df['release_date'].replace({'no release date' : ''})
df['game_title'] = df['game_title'].str.replace(r'®', '')
df['game_title'] = df['game_title'].str.replace(r'™', '')
df['number_of_review'] = df['number_of_review'].replace({'no reviews' : ''})
df['score'] = df['score'].replace({'no rating' : ''})

# Dropping unneeded/dupe columns and renaming others for merge purposes later
df = df.drop(['app'], axis=1)
df['game_ID'] = pd.to_numeric(df['game_ID'], downcast='signed')
df['number_of_review'] = pd.to_numeric(df['number_of_review'], downcast='integer')
df['score'] = pd.to_numeric(df['score'])

# Game must have a score, scores are not calculated until a min number of reviews is satisfied
df = df[df['score'] > 0]
df = df.rename(columns={"game_ID": "appid"})
df = df.rename(columns={"Unnamed: 0": "id"})
df = df.drop_duplicates(subset='appid', keep='first')
df = df.dropna(subset=['game_title'])
df = df.reset_index()
df = df.drop(['index'], axis=1)

In [5]:
text = " ".join(desc for desc in df.tags)
textlist = text.split(", ")
wordfreq = [textlist.count(w) for w in textlist]
pairs = Counter(text.split(", ")).most_common(25)
word = []
frequency = []

for i in range(len(pairs)):
    word.append(pairs[i][0])
    frequency.append(pairs[i][1])

indices = np.arange(len(pairs))

NameError: name 'Counter' is not defined